In [36]:
# importing necessary modules.
import requests
import json
import pandas as pd
import numpy as np

In [37]:
# importing csv that contains the stock ticker symbols and names of the S&P 500.
sandp_df = pd.read_csv('../Resources/tickers_SandP_500.csv')
russell_2000_df = pd.read_csv('../Resources/tickers_russell_2000.csv')

In [38]:
# creating empty list.
ticker_symbols = []

# looping through above csv and appending stock ticker symbols to the empty list.
for ticker in sandp_df['Ticker']:
    ticker_symbols.append(ticker)

# creating empty list.
names = []  

# looping through csv and appending names to the empty list.
for name in sandp_df['Name']:
    names.append(name)

In [39]:
# creating/initializing an empty pandas dataframe.
dataframe2 = pd.DataFrame()

# creating loop that goes through the ticker_symbols and names lists created above.
for (ticker, name) in zip(ticker_symbols, names):
    
    # creating a for loop that goes through the last two entry's value of the json dictionaries from financialmodelingprep.    
    for i in range(-2,0):
        
        # setting up try and except statement to avoid lack of financialmodelingprep json data to ticker_symbol list.         
        try:
            
            # obtaining the json data for income statement from financialmodelingprep.          
            response = requests.get(f'https://financialmodelingprep.com/api/v3/financials/income-statement/{ticker}').json()
            
            print(f'Processing:{ticker} {name}')
            
            # creating empty list named columns.             
            columns = []
            
            # appending 'symbol' to columns list.                
            columns.append('symbol')
            
            # appending 'name' to columns list.            
            columns.append('name')
        
            # appending 'date' to columns list.         
            columns.append('date')
            
            # creating empty list named data.            
            data = []
            
            # filtering the the financial json dicionary data by years 2009 and 2010.            
            if response['financials'][i]['date'].split('-')[0] == '2009' or response['financials'][i]['date'].split('-')[0] == '2010':
                
                # creating another try except statement to avoid pitfalls.                
                try:
                    
                    # appending symbol to data list.
                    data.append(ticker)
                   
                    # appending company name to data list.
                    data.append(name)
                    
                    # appending date to data list.
                    data.append(response['financials'][i]['date'])
                    
                    # saving financial ratio dicionary to variable.
                    keys = response['financials'][i]
                    
                    # looping through income statement keys.                   
                    for key in keys:
                        
                        # avoiding date key.                       
                        if key != 'date':

                            #  appending income statement dict keys to columns list.                                
                            columns.append(key)

                            # filtering income statement values that are blank.                 
                            if keys[key] == '':

                                # appending 0 in the spot of nothing to the data list.                                     
                                data.append(0)

                            # continuing the filtering process, but with some values.         
                            else:

                                # appending those financial ratio values to data list.                                 
                                data.append(float(keys[key]))
                    
                    # creating a connection and obtaining balance sheet statistics from financialmodelingprep.
                    response2 = requests.get(f'https://financialmodelingprep.com/api/v3/financials/balance-sheet-statement/{ticker}').json()
                    
                    # saving financial growth dict to variable.                  
                    keys2 = response2['financials'][i]
                    
                    # creating for loop to go through balance sheet. 
                    for key in keys2:
                        
                        # avoiding the date key.                       
                        if key != 'date':
                                                        
                            # appending the balance sheet keys to columns list.                            
                            columns.append(key)

                            # filtering balance sheet values that have nan values.
                            if keys2[key] == '':

                                # replacing nan values with 0.                                 
                                data.append(0)

                            # filtering balance sheet values that are not nan.
                            else:

                                # appending those balance sheet values to the data list.        
                                data.append(float(keys2[key]))
                                
                    # creating a connection and obtaining cash flow statistics from financialmodelingprep.
                    response3 = requests.get(f'https://financialmodelingprep.com/api/v3/financials/cash-flow-statement/{ticker}').json()
                    
                    # saving cash flow  dict to variable.                  
                    keys3 = response3['financials'][i]
                    
                    # creating for loop to go through cash flow. 
                    for key in keys3:
                        
                        # avoiding the date key.                       
                        if key != 'date':

                            # appending the cash flow  keys to columns list.                            
                            columns.append(key)

                            # filtering cash flow  values that have nan values.
                            if keys3[key] == '':

                                # replacing nan values with 0.                                 
                                data.append(0)

                            # filtering cash flow  values that are not nan.
                            else:

                                # appending those cash flow  values to the data list.        
                                data.append(float(keys3[key]))

                                
                    # obtaining the json data for market cap from financialmodelingprep.   
                    response4 = requests.get(f'https://financialmodelingprep.com/api/v3/enterprise-value/{ticker}').json()
                    
                    
                    # filtering share price values that are nan.
                    if response4['enterpriseValues'][i]['Market Capitalization'] == '':

                        # setting returns variable to 0 based on no share price data.                    
                        market_cap = 0

                    else:

                        # calculating average market cap value from 2009/2010.             
                        market_cap = (int(response4['enterpriseValues'][-1]['Market Capitalization']) + int(response4['enterpriseValues'][-2]['Market Capitalization']))/2


                    # creating a connection to get the company key metrics data from financialmodelingprep and putting in json form.
                    response5 = requests.get(f'https://financialmodelingprep.com/api/v3/enterprise-value/{ticker}').json()
                    
                    # filtering share price values that are nan.
                    if response5['enterpriseValues'][i]['Stock Price'] == '':
                        
                        # setting returns variable to 0 based on no share price data.                    
                        returns = 0
                    
                    # filtering chare price based on having a value.
                    else:
                        
                        # using share price of 2009 and 2018/2019 to calculated percentage return over the time span.                        
                        returns = ((((float(response5['enterpriseValues'][0]['Stock Price']) + float(response5['enterpriseValues'][1]['Stock Price']))/2) - ((float(response5['enterpriseValues'][-2]['Stock Price']) + float(response5['enterpriseValues'][-1]['Stock Price']))/2))/((float(response5['enterpriseValues'][-2]['Stock Price']) + float(response5['enterpriseValues'][-1]['Stock Price']))/2))*100
                    
                    # creating a pandas dataframe based on the data and column lists developed through the above process.
                    dataframe = pd.DataFrame([data], columns=columns)
                    
                    dataframe['market_cap'] = [market_cap]
                    
                    # appending a share price return on investment to newly created dataframe.
                    dataframe['% return on investment'] = [returns]
                    
                    # appending the dataframe to the empty dataframe created at the very top.
                    dataframe2 = dataframe2.append(dataframe)
                
                # finishing try except statement.
                except:
                    
                    # telling program to move on and ignore pitfall.                    
                    pass
            
            # finishing date(2009,2010) if then statement created near the bottom of the top.         
            else:
                
                 # telling program to move on and ignore pitfall.
                pass
        
        # finishing the other try except statement.        
        except:
            
             # telling program to move on and ignore pitfall.
            pass

# grouping completed dataframe by symbol and name and finding the average of the columns, because used multiple dates.
dataframe2 = dataframe2.groupby(['symbol', 'name']).mean()

# viewing dataframe.
dataframe2

Processing:A Agilent Technologies, Inc.
Processing:A Agilent Technologies, Inc.
Processing:AAL American Airlines Group, Inc.
Processing:AAL American Airlines Group, Inc.
Processing:AAP Advance Auto Parts, Inc.
Processing:AAP Advance Auto Parts, Inc.
Processing:AAPL Apple, Inc.
Processing:AAPL Apple, Inc.
Processing:ABBV AbbVie, Inc.
Processing:ABBV AbbVie, Inc.
Processing:ABC AmerisourceBergen Corp.
Processing:ABC AmerisourceBergen Corp.
Processing:ABMD ABIOMED, Inc.
Processing:ABMD ABIOMED, Inc.
Processing:ABT Abbott Laboratories
Processing:ABT Abbott Laboratories
Processing:ACN Accenture Plc
Processing:ACN Accenture Plc
Processing:ADBE Adobe, Inc.
Processing:ADBE Adobe, Inc.
Processing:ADI Analog Devices, Inc.
Processing:ADI Analog Devices, Inc.
Processing:ADM Archer-Daniels-Midland Co.
Processing:ADM Archer-Daniels-Midland Co.
Processing:ADP Automatic Data Processing, Inc.
Processing:ADP Automatic Data Processing, Inc.
Processing:ADS Alliance Data Systems Corp.
Processing:ADS Allian

Processing:COP ConocoPhillips
Processing:COP ConocoPhillips
Processing:COST Costco Wholesale Corp.
Processing:COST Costco Wholesale Corp.
Processing:COTY Coty, Inc.
Processing:COTY Coty, Inc.
Processing:CPB Campbell Soup Co.
Processing:CPB Campbell Soup Co.
Processing:CPRI Capri Holdings Ltd.
Processing:CPRI Capri Holdings Ltd.
Processing:CPRT Copart, Inc.
Processing:CPRT Copart, Inc.
Processing:CRM salesforce.com, inc.
Processing:CRM salesforce.com, inc.
Processing:CSCO Cisco Systems, Inc.
Processing:CSCO Cisco Systems, Inc.
Processing:CSX CSX Corp.
Processing:CSX CSX Corp.
Processing:CTL CenturyLink, Inc.
Processing:CTL CenturyLink, Inc.
Processing:CTSH Cognizant Technology Solutions Corp.
Processing:CTSH Cognizant Technology Solutions Corp.
Processing:CTVA Corteva, Inc.
Processing:CTVA Corteva, Inc.
Processing:CTXS Citrix Systems, Inc.
Processing:CTXS Citrix Systems, Inc.
Processing:CVS CVS Health Corp.
Processing:CVS CVS Health Corp.
Processing:CVX Chevron Corp.
Processing:CVX Chev

Processing:HPE Hewlett-Packard Enterprise Co.
Processing:HPE Hewlett-Packard Enterprise Co.
Processing:HPQ HP, Inc.
Processing:HPQ HP, Inc.
Processing:HRB H&R Block, Inc.
Processing:HRB H&R Block, Inc.
Processing:HRL Hormel Foods Corp.
Processing:HRL Hormel Foods Corp.
Processing:HSIC Henry Schein, Inc.
Processing:HSIC Henry Schein, Inc.
Processing:HST Host Hotels & Resorts, Inc.
Processing:HST Host Hotels & Resorts, Inc.
Processing:HSY The Hershey Co.
Processing:HSY The Hershey Co.
Processing:HUM Humana, Inc.
Processing:HUM Humana, Inc.
Processing:IBM International Business Machines Corp.
Processing:IBM International Business Machines Corp.
Processing:ICE Intercontinental Exchange, Inc.
Processing:ICE Intercontinental Exchange, Inc.
Processing:IDXX IDEXX Laboratories, Inc.
Processing:IDXX IDEXX Laboratories, Inc.
Processing:IEX IDEX Corp.
Processing:IEX IDEX Corp.
Processing:IFF International Flavors & Fragrances, Inc.
Processing:IFF International Flavors & Fragrances, Inc.
Processing

Processing:NTAP NetApp, Inc.
Processing:NTAP NetApp, Inc.
Processing:NTRS Northern Trust Corp.
Processing:NTRS Northern Trust Corp.
Processing:NUE Nucor Corp.
Processing:NUE Nucor Corp.
Processing:NVDA NVIDIA Corp.
Processing:NVDA NVIDIA Corp.
Processing:NVR NVR, Inc.
Processing:NVR NVR, Inc.
Processing:NWL Newell Brands, Inc.
Processing:NWL Newell Brands, Inc.
Processing:NWS News Corp.
Processing:NWS News Corp.
Processing:NWSA News Corp.
Processing:NWSA News Corp.
Processing:O Realty Income Corp.
Processing:O Realty Income Corp.
Processing:OKE ONEOK, Inc.
Processing:OKE ONEOK, Inc.
Processing:OMC Omnicom Group, Inc.
Processing:OMC Omnicom Group, Inc.
Processing:ORCL Oracle Corp.
Processing:ORCL Oracle Corp.
Processing:ORLY O'Reilly Automotive, Inc.
Processing:ORLY O'Reilly Automotive, Inc.
Processing:OXY Occidental Petroleum Corp.
Processing:OXY Occidental Petroleum Corp.
Processing:PAYX Paychex, Inc.
Processing:PAYX Paychex, Inc.
Processing:PBCT People's United Financial, Inc.
Proces

Processing:USB U.S. Bancorp
Processing:UTX United Technologies Corp.
Processing:UTX United Technologies Corp.
Processing:V Visa, Inc.
Processing:V Visa, Inc.
Processing:VAR Varian Medical Systems, Inc.
Processing:VAR Varian Medical Systems, Inc.
Processing:VFC VF Corp.
Processing:VFC VF Corp.
Processing:VIAB Viacom Inc.
Processing:VIAB Viacom Inc.
Processing:VLO Valero Energy Corp.
Processing:VLO Valero Energy Corp.
Processing:VMC Vulcan Materials Co.
Processing:VMC Vulcan Materials Co.
Processing:VNO Vornado Realty Trust
Processing:VNO Vornado Realty Trust
Processing:VRSK Verisk Analytics, Inc.
Processing:VRSK Verisk Analytics, Inc.
Processing:VRSN VeriSign, Inc.
Processing:VRSN VeriSign, Inc.
Processing:VRTX Vertex Pharmaceuticals, Inc.
Processing:VRTX Vertex Pharmaceuticals, Inc.
Processing:VTR Ventas, Inc.
Processing:VTR Ventas, Inc.
Processing:VZ Verizon Communications, Inc.
Processing:VZ Verizon Communications, Inc.
Processing:WAB Westinghouse Air Brake Technologies Corp.
Process

,,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,...,Issuance (repayment) of debt,Issuance (buybacks) of shares,Dividend payments,Financing Cash Flow,Effect of forex changes on cash,Net cash flow / Change in cash,Free Cash Flow,Net Cash/Marketcap,market_cap,% return on investment
symbol,name,,,,,,,,,,,,,,,,,,,,,
A,"Agilent Technologies, Inc.",4.962500e+09,-0.004496,2.351500e+09,2.611000e+09,6.270000e+08,1.677500e+09,2.304500e+09,3.065000e+08,9.200000e+07,3.495000e+08,...,7.280000e+08,-9.900000e+07,0.000000e+00,6.290000e+08,24000000.0,6.220000e+08,4.425000e+08,0.12275,7.462700e+09,215.407785
AAL,"American Airlines Group, Inc.",9.958500e+09,-0.581000,5.015500e+09,4.943000e+09,0.000000e+00,3.403500e+09,5.445000e+09,-5.020000e+08,3.720000e+08,-8.760000e+08,...,2.870000e+08,2.065000e+08,0.000000e+00,8.775000e+08,0.0,-1.900000e+07,-2.575000e+08,-2.12480,6.133809e+08,492.976589
AAP,"Advance Auto Parts, Inc.",5.412623e+09,0.052600,2.768397e+09,2.644226e+09,0.000000e+00,2.189841e+09,2.189841e+09,4.543850e+08,2.333700e+07,4.316550e+08,...,-2.625600e+08,-6.466000e+07,-2.280300e+07,-4.514910e+08,0.0,6.266000e+07,5.141510e+08,-0.02730,1.912092e+09,158.290852
AAPL,"Apple, Inc.",5.406500e+10,0.332310,3.261200e+10,2.145300e+10,1.557500e+09,4.833000e+09,6.390500e+09,1.506250e+10,0.000000e+00,1.530300e+10,...,0.000000e+00,-4.937000e+09,0.000000e+00,9.600000e+08,0.0,-3.070000e+08,1.276800e+10,0.07035,9.151793e+10,641.441749
ABBV,"AbbVie, Inc.",1.492610e+10,0.050084,4.174695e+09,1.075140e+10,2.101220e+09,3.584286e+09,5.927006e+09,4.824397e+09,-1.400000e+07,5.393158e+09,...,-1.738300e+07,0.000000e+00,0.000000e+00,-2.408076e+09,0.0,4.915000e+06,4.791386e+09,-0.13870,6.619065e+10,127.493713
ABC,AmerisourceBergen Corp.,7.485698e+10,0.054358,7.262863e+10,2.228358e+09,0.000000e+00,1.144034e+09,1.200306e+09,9.936920e+08,2.915350e+07,9.216940e+08,...,-1.042200e+07,-4.493200e+08,-7.665900e+07,-3.897995e+08,0.0,3.900340e+08,7.810115e+08,-0.01265,7.017275e+09,258.118955
ABMD,"ABIOMED, Inc.",8.571300e+07,0.170800,2.252900e+07,6.318400e+07,2.595400e+07,6.083700e+07,8.826000e+07,-2.507600e+07,0.000000e+00,-1.835300e+07,...,0.000000e+00,9.790000e+05,0.000000e+00,9.790000e+05,568000.0,3.003000e+06,-9.162000e+06,0.15060,4.310088e+08,2583.734150
ABT,Abbott Laboratories,3.296571e+10,0.092493,1.409386e+10,1.887185e+10,3.234078e+09,9.391114e+09,1.271019e+10,6.161661e+09,5.363955e+08,5.909973e+09,...,1.030078e+09,-5.922505e+08,-2.542968e+09,-4.286100e+07,-251022500.0,-2.318255e+08,6.953509e+09,-0.08140,3.765926e+10,167.453799
ACN,Accenture Plc,2.313252e+10,-0.044009,1.531961e+10,7.812916e+09,1.918500e+08,4.137170e+09,5.033562e+09,2.779355e+09,1.439900e+07,2.621961e+09,...,-3.828000e+06,-1.760774e+09,-6.012970e+08,-2.139664e+09,-109011000.0,6.177660e+08,2.887408e+09,0.09655,2.405644e+10,363.726903


In [40]:
# filtering the completed dataframe by creating a new dataframe that the does not have the rows associated with 0 in the % return on investment column.
dataframe2 = dataframe2[dataframe2['% return on investment'] != 0]

In [42]:
dataframe2.to_csv('new_route_df.csv')